In [113]:
#importam lib necesare
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
import numpy as np
import pandas as pd
from sklearn.model_selection import cross_validate, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import make_scorer, f1_score, roc_auc_score, accuracy_score

In [121]:

#Tema 3 - Clasificare

#modificam parametrii algoritmului pentru a observa efectul asupra predictiilor
#evaluarea modelului se realizeaza folosind strategia de 10-fold cross-validation
#utilizam cel putin 3 metrici pentru a estima performanta modelului

#initializam clasificatorul
clf = RandomForestClassifier(n_estimators=100, random_state=42)

#selectam coloanele folosite pentru clasificare si target-ul
X = data.drop('G3', axis=1)  #'G3' este coloana target
y = data['G3'] > 10  #transformam problema intr-o clasificare binara

#evaluam modelul folosind cross-validation
scores = cross_val_score(clf, X, y, cv=10)
print("Acuratete medie:", scores.mean())


Acuratete medie: 1.0


In [124]:
data = pd.read_csv("student-mat.csv")
#pregatim datele pentru clasificare
#convertim G3 intr-o variabila binara conform cerintelor
#daca G3 este mai mare sau egal cu 10, spunem ca elevul a trecut (1), altfel nu (0)
data['G3_binary'] = (data['G3'] >= 10).astype(int)

#alegem caracteristicile (X) si tinta (y)
#ne uitam la varsta elevului, cate absente are si prima nota (G1)
X = data[['age', 'absences', 'G1']]
y = data['G3_binary']  #variabila tinta binara

#definim functia pentru masurarea scorului ROC AUC (cat de bine separa clasele)
#aceasta masura este utila mai ales cand clasele sunt dezechilibrate (mai multi elevi care trec decat cei care pica)
def safe_roc_auc_scorer(y_true, y_pred_proba):
    #verifica daca y_pred_proba este unidimensional
    if y_pred_proba.ndim == 1 or y_pred_proba.shape[1] == 1:
        y_pred_proba = np.column_stack((1 - y_pred_proba, y_pred_proba))
    return roc_auc_score(y_true, y_pred_proba[:, 1])

#creare scorer pentru ROC AUC
#alegem sa folosim un scorer personalizat pentru a ne asigura ca masuram ROC AUC corect, chiar daca modelul returneaza o predictie unidimensionala
roc_auc_scorer = make_scorer(safe_roc_auc_scorer, needs_proba=True)

#definim scorurile pentru evaluare
scoring = {
    'accuracy': 'accuracy', #pentru a vedea procentul total al predictiilor corecte
    'f1': make_scorer(f1_score, average='binary', zero_division=1), #pentru a echilibra precizia si rechemarea, mai ales util in cazul claselor dezechilibrate
    'roc_auc': roc_auc_scorer #pentru a vedea cat de bine separa modelul cele doua clase
}

#configuram validarea incrucisata
#alegem StratifiedKFold pentru a pastra proportiile claselor in fiecare fold, evitand astfel evaluari incorecte in cazul datelor dezechilibrate
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

#initializam clasificatorul
#Random Forest foloseste 100 de arbori de decizie cu o adancime maxima de 10
clf = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)

#evaluam modelul folosind cross_validate
#validam modelul pe 10 parti diferite pentru a preveni overfitting-ul si calculam scorurile
results = cross_validate(clf, X, y, cv=cv, scoring=scoring, return_train_score=False)

#calculam media acuratetei, F1 Score si ROC AUC din cele 10 fold-uri
#afisam scorurile medii pentru fiecare metrica
print("Accuracy:", np.mean(results['test_accuracy']))
print("F1 Score:", np.mean(results['test_f1']))
print("ROC AUC:", np.mean(results['test_roc_auc']))


C:\Users\Andreea\miniconda3\envs\RNP\Lib\site-packages\sklearn\metrics\_scorer.py:610: FutureWarning: The `needs_threshold` and `needs_proba` parameter are deprecated in version 1.4 and will be removed in 1.6. You can either let `response_method` be `None` or set it to `predict` to preserve the same behaviour.
  warnings.warn(


Accuracy: 0.8382051282051283
F1 Score: 0.8800294944512658
ROC AUC: 0.9274819197896121
